<a href="https://colab.research.google.com/github/sohamk1112/AbstractiveTextSummarizer/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [ ]:
# initialize the pretrained model
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# input text
text = """
Artificial Intelligence (AI) is a multidisciplinary field of computer science and engineering that seeks to create intelligent systems capable of mimicking human cognitive functions. At its core, AI is the science of developing algorithms, models, and systems that can perform tasks that typically require human intelligence. These tasks encompass a wide range of activities, from problem-solving and decision-making to understanding natural language, recognizing patterns, and learning from data.

One of the foundational principles of AI is the concept of machine learning, wherein AI systems can improve their performance on a specific task through exposure to data and experience, rather than relying solely on explicit programming. Machine learning algorithms enable AI systems to identify patterns, make predictions, and adapt to changing conditions, making them incredibly versatile and applicable across various domains.

AI can be categorized into two broad types:

Narrow or Weak AI: This type of AI is designed to excel at specific tasks or domains. Examples include voice assistants like Siri or recommendation systems on streaming platforms. While they may seem intelligent within their specialized scope, they lack general intelligence and self-awareness.

General or Strong AI: This represents the aspiration of creating AI systems with human-level intelligence and cognitive abilities. General AI would possess the capacity to understand, learn, and adapt across a wide range of tasks and contexts, similar to human intelligence. Achieving true general AI remains a long-term goal and a subject of ongoing research.

AI technologies encompass a diverse set of techniques and approaches, including neural networks, natural language processing, computer vision, robotics, and reinforcement learning, among others. These tools find applications across numerous sectors, such as healthcare (diagnosis and treatment optimization), finance (algorithmic trading and fraud detection), transportation (autonomous vehicles), and entertainment (video game AI and content recommendation).

While AI holds immense promise, it also raises ethical and societal questions related to privacy, bias, transparency, and the potential impact on the workforce. As AI continues to advance, it is crucial to ensure responsible development and deployment, guided by ethical principles and a commitment to harnessing its potential for the betterment of society. In essence, AI represents a powerful and transformative force in the modern world, with the potential to reshape industries, improve efficiency, and enhance our understanding of intelligence and cognition.
"""

In [ ]:
## preprocess the input text
preprocessed_text = text.strip().replace('\n','')
t5_input_text = 'summarize: ' + preprocessed_text

In [ ]:
t5_input_text

'summarize: Artificial Intelligence (AI) is a multidisciplinary field of computer science and engineering that seeks to create intelligent systems capable of mimicking human cognitive functions. At its core, AI is the science of developing algorithms, models, and systems that can perform tasks that typically require human intelligence. These tasks encompass a wide range of activities, from problem-solving and decision-making to understanding natural language, recognizing patterns, and learning from data.One of the foundational principles of AI is the concept of machine learning, wherein AI systems can improve their performance on a specific task through exposure to data and experience, rather than relying solely on explicit programming. Machine learning algorithms enable AI systems to identify patterns, make predictions, and adapt to changing conditions, making them incredibly versatile and applicable across various domains.AI can be categorized into two broad types:Narrow or Weak AI: 

In [ ]:
len(t5_input_text.split())

370

In [ ]:
tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=512).to(device)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
summary_ids = model.generate(tokenized_text, min_length=30, max_length=120)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
summary

'AI is a multidisciplinary field of computer science and engineering. it seeks to create intelligent systems capable of mimicking human cognitive functions. these tasks encompass a wide range of activities, from problem-solving to decision-making.'

In [ ]:
!pip install rouge

BLEU and ROUGE score of T5

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from rouge import Rouge

# Example of reference and generated summaries
reference_summary = ["Artificial Intelligence (AI) is a multidisciplinary field aiming to create intelligent systems mimicking human cognitive functions, driven by machine learning. It holds promise for transforming industries but raises ethical concerns regarding privacy and workforce impact."]
generated_summary = ["AI is a multidisciplinary field of computer science and engineering. it seeks to create intelligent systems capable of mimicking human cognitive functions. these tasks encompass a wide range of activities, from problem-solving to decision-making."]

# Initialize ROUGE object
rouge = Rouge()

# Calculate ROUGE scores
scores = rouge.get_scores(generated_summary, reference_summary)

# Print ROUGE scores
print("ROUGE Scores:")
print(scores)

ROUGE Scores:
[{'rouge-1': {'r': 0.34285714285714286, 'p': 0.4, 'f': 0.36923076426035506}, 'rouge-2': {'r': 0.23529411764705882, 'p': 0.24242424242424243, 'f': 0.2388059651503677}, 'rouge-l': {'r': 0.34285714285714286, 'p': 0.4, 'f': 0.36923076426035506}}]


In [ ]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Example of reference and generated summaries
reference_summary = [["Artificial Intelligence (AI) is a multidisciplinary field aiming to create intelligent systems mimicking human cognitive functions, driven by machine learning. It holds promise for transforming industries but raises ethical concerns regarding privacy and workforce impact."]]
generated_summary = ["AI is a multidisciplinary field of computer science and engineering. it seeks to create intelligent systems capable of mimicking human cognitive functions. these tasks encompass a wide range of activities, from problem-solving to decision-making."]

# Smoothie method to handle short reference length
smoothie = SmoothingFunction().method1

# Calculate BLEU score
bleu_score = corpus_bleu(reference_summary, generated_summary, smoothing_function=smoothie)

# Print BLEU score
print("BLEU Score:", bleu_score)


BLEU Score: 0.5330754663254875
